# Libraries

In [48]:
#https://www.datacamp.com/community/tutorials/fuzzy-string-python

import pandas as pd
import pyreadstat
from fuzzywuzzy import fuzz
from matplotlib import pyplot as plt
pd.options.mode.chained_assignment = None

# Read Data

In [2]:
londonCodeDF = pd.read_csv("LondonCodes.csv")
wardDataDemoDF = pd.read_table("WardDataDemo.dat")
wardDataEnviroDF = pd.read_csv("WardDataEnvironment.csv")
wardDataHealthDF = pd.read_sas("WardDataHealth.sas7bdat", format='sas7bdat', encoding="latin1")
wardDataSocDF, meta = pyreadstat.read_sav("WardDataSoc.sav")

In [3]:
FullDF = pd.DataFrame(columns=['Wardcode', 'District', 'Districtcode'])

### London Dataset 

In [4]:
londonCodeDF.head()

,District,Districtcode
0,Barking and Dagenham,00AB
1,Barnet,00AC
2,Bexley,00AD
3,Brent,00AE
4,Bromley,00AF


### Merge on ward name

In [21]:
wardDataDemoDF.head()

,Wardname,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking
0,Hackney - Queensbridge,17.542063,8.2,44.7,38.3,11.8
1,Hammersmith & Fulham - S&s End,17.915361,8.4,30.6,39.0,11.8
2,Barking & Dagenham - River,26.851598,10.1,38.4,30.9,9.6
3,Tower Hamlets - Bethnal Green North,18.555872,7.0,49.4,38.7,15.0
4,Merton - Abbey,15.731861,8.5,26.6,38.3,12.5


In [22]:
wardDataHealthDF.head()

,Wardname,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy
0,Bromley - Darwin,5110.0,63.8,81.2,82.4
1,Kensington and Chelsea - Royal Hospital,7252.0,52.3,80.5,89.6
2,Hillingdon - Harefield,7399.0,55.8,78.3,82.3
3,Hammersmith and Fulham - Palace Riverside,7483.0,40.0,80.9,89.0
4,Kensington and Chelsea - Pembridge,7659.0,40.4,82.1,85.7


### Merge on wardcode

In [7]:
wardDataEnviroDF.head()

,Wardcode,Population2011Census,Crimerate,Openspace
0,00ANGQ,11201,117.7,0.0
1,00ANGA,11518,114.0,0.3
2,00ADGN,10800,44.2,0.7
3,00BEGH,12321,65.3,0.7
4,00BCFZ,12609,64.3,1.3


In [8]:
wardDataSocDF.head()

,Wardcode,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,00ABFX,26.7,8.7,16.4,0.5
1,00ABFY,36.8,10.2,31.2,0.8
2,00ABFZ,29.4,10.2,28.0,0.9
3,00ABGB,20.0,8.8,29.9,1.1
4,00ABGC,37.4,10.2,28.9,0.8


### Fuzzy Matching Method 

In [9]:
def get_match(Str1, Str2):
    Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
    return Ratio

def get_P_match(Str1, Str2):
    Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
    return Partial_Ratio

###  Merge the two wardcode tables

In [10]:
columns = ['Wardcode', 'Population2011Census', 'Crimerate', 'Openspace', 'hhSocialRented', 'JobSeekers', 'Noqual','Carsperhousehold']
FullWardDataDF = pd.DataFrame(columns=columns)
for i in range(0,len(wardDataEnviroDF)):
    for j in range(0,len(wardDataSocDF)):
        if(get_match( wardDataEnviroDF['Wardcode'][i], wardDataSocDF['Wardcode'][j]) > 90):
            FullWardDataDF.loc[len(FullWardDataDF)] = [wardDataEnviroDF['Wardcode'][i], wardDataEnviroDF['Population2011Census'][i],wardDataEnviroDF['Crimerate'][i], wardDataEnviroDF['Openspace'][i],wardDataSocDF['hhSocialRented'][j],wardDataSocDF['JobSeekers'][j],wardDataSocDF['Noqual'][j],wardDataSocDF['Carsperhousehold'][j]]
            #print (str(i)+ " " + str(j))

In [23]:
FullWardDataDF.head()

,Wardcode,Population2011Census,Crimerate,Openspace,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,00ANGQ,11201,117.7,0.0,23.7,3.6,9.4,0.6
1,00ANGA,11518,114.0,0.3,24.9,4.5,9.3,0.5
2,00ADGN,10800,44.2,0.7,1.1,2.9,22.1,1.3
3,00BEGH,12321,65.3,0.7,20.3,4.6,12.3,0.7
4,00BCFZ,12609,64.3,1.3,4.7,4.2,17.2,1.2


### Merge the two wardname tables 

In [12]:
columns = ['Wardname','Children','Greaterthan65','nonwhite','NotBorninUK','NotEnglishspeaking', 'Population2011Census','GeneralFertilityRate','Malelifeexpectancy','Femalelifeexpectancy']
FullHealthDataDF = pd.DataFrame(columns=columns)

for i in range(0,len(wardDataDemoDF)):
    for j in range(0,len(wardDataHealthDF)):
        if(get_match( str(wardDataDemoDF['Wardname'][i]), str(wardDataHealthDF['Wardname'][j])) > 93):
            FullHealthDataDF.loc[len(FullHealthDataDF)] = [wardDataDemoDF['Wardname'][i], wardDataDemoDF['Children'][i],wardDataDemoDF['Greaterthan65'][i], wardDataDemoDF['nonwhite'][i],wardDataDemoDF['NotBorninUK'][i],wardDataDemoDF['NotEnglishspeaking'][i],wardDataHealthDF['Population2011Census'][j],wardDataHealthDF['GeneralFertilityRate'][j],wardDataHealthDF['Malelifeexpectancy'][j],wardDataHealthDF['Femalelifeexpectancy'][j]]
            break


In [128]:
FullHealthDataDF.head()

,Wardname,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy
0,Hackney - Queensbridge,17.542063,8.2,44.7,38.3,11.8,13670.0,69.8,75.9,82.6
1,Tower Hamlets - Bethnal Green North,18.555872,7.0,49.4,38.7,15.0,13683.0,54.5,73.6,78.6
2,Merton - Abbey,15.731861,8.5,26.6,38.3,12.5,10323.0,52.6,80.0,81.0
3,Richmond upon Thames - South Richmond,16.552680,14.0,13.1,33.0,8.5,10649.0,76.4,83.8,86.1
4,Lambeth - Larkhall,16.656034,6.2,43.3,42.2,13.0,17243.0,38.8,74.0,79.2


### Make Final DF 

In [28]:
CompleteDF = pd.DataFrame(columns=['Ward_Name','District','Districtcode'])

for i in range(0,len(FullHealthDataDF)):
    for j in range(0,len(londonCodeDF)):
        if(get_P_match( str(FullHealthDataDF['Wardname'][i]), str(londonCodeDF['District'][j])) > 90):
            CompleteDF.loc[len(CompleteDF)] = [FullHealthDataDF['Wardname'][i], londonCodeDF['District'][j],londonCodeDF['Districtcode'][j]]
            break



In [29]:
CompleteDF.head()

,Ward_Name,District,Districtcode
0,Hackney - Queensbridge,Hackney,00AM
1,Tower Hamlets - Bethnal Green North,Tower Hamlets,00BG
2,Merton - Abbey,Merton,00BA
3,Richmond upon Thames - South Richmond,Richmond upon Thames,00BD
4,Lambeth - Larkhall,Lambeth,00AY
5,Greenwich - Plumstead,Greenwich,00AL
6,Hillingdon - Heathrow Villages,Hillingdon,00AS
7,Lambeth - Streatham Hill,Lambeth,00AY
8,Havering - Gooshays,Havering,00AR
9,Croydon - Broad Green,Croydon,00AH


In [30]:
FullWardDataDF.head()

,Wardcode,Population2011Census,Crimerate,Openspace,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,00ANGQ,11201,117.7,0.0,23.7,3.6,9.4,0.6
1,00ANGA,11518,114.0,0.3,24.9,4.5,9.3,0.5
2,00ADGN,10800,44.2,0.7,1.1,2.9,22.1,1.3
3,00BEGH,12321,65.3,0.7,20.3,4.6,12.3,0.7
4,00BCFZ,12609,64.3,1.3,4.7,4.2,17.2,1.2


In [31]:
CompleteDF['Population2011Census'] = None
CompleteDF['Crimerate'] = None
CompleteDF['Openspace'] = None
CompleteDF['hhSocialRented'] = None
CompleteDF['JobSeekers'] = None
CompleteDF['Noqual'] = None
CompleteDF['Carsperhousehold'] = None

for i in range(0,len(FullWardDataDF)):
    for j in range(0,len(CompleteDF)):
        if(get_match( str(FullWardDataDF['Wardcode'][i][:-2]), str(CompleteDF['Districtcode'][j]).strip()) > 90):
            CompleteDF.loc[j,'Population2011Census'] = FullWardDataDF['Population2011Census'][i]
            CompleteDF.loc[j,'Crimerate'] = FullWardDataDF['Crimerate'][i]
            CompleteDF.loc[j,'Openspace'] = FullWardDataDF['Openspace'][i]
            CompleteDF.loc[j,'hhSocialRented'] = FullWardDataDF['hhSocialRented'][i]
            CompleteDF.loc[j,'JobSeekers'] = FullWardDataDF['JobSeekers'][i]
            CompleteDF.loc[j,'Noqual'] = FullWardDataDF['Noqual'][i]
            CompleteDF.loc[j,'Carsperhousehold'] = FullWardDataDF['Carsperhousehold'][i]

In [32]:
CompleteDF.rename(columns={'Ward_Name': 'Wardname'}, inplace=True)
CompleteDF = pd.merge(CompleteDF, FullHealthDataDF, on='Wardname')

In [40]:
# Calculate Joint Life expectancy
def newlife(row):
    return (row['Malelifeexpectancy'] + row['Femalelifeexpectancy'])/2

In [42]:
CompleteDF['LifeTotal'] = CompleteDF.apply(newlife, axis=1)

In [58]:
CompleteDF

,Wardname,District,Districtcode,Population2011Census_x,Crimerate,Openspace,hhSocialRented,JobSeekers,Noqual,Carsperhousehold,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,Population2011Census_y,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy,LifeTotal
0,Hackney - Queensbridge,Hackney,00AM,11098,61.9,67.7,57.9,13.6,24.6,0.5,17.542063,8.2,44.7,38.3,11.8,13670.0,69.8,75.9,82.6,79.25
1,Tower Hamlets - Bethnal Green North,Tower Hamlets,00BG,12939,145.3,43,39,8.8,20.3,0.5,18.555872,7.0,49.4,38.7,15.0,13683.0,54.5,73.6,78.6,76.10
2,Merton - Abbey,Merton,00BA,8491,60.3,62.2,5.6,0.9,6.8,1.4,15.731861,8.5,26.6,38.3,12.5,10323.0,52.6,80.0,81.0,80.50
3,Richmond upon Thames - South Richmond,Richmond upon Thames,00BD,10317,65.8,81.9,18.5,3.1,14.1,1,16.552680,14.0,13.1,33.0,8.5,10649.0,76.4,83.8,86.1,84.95
4,Lambeth - Larkhall,Lambeth,00AY,15107,133.6,37.9,31.8,8.5,12.2,0.5,16.656034,6.2,43.3,42.2,13.0,17243.0,38.8,74.0,79.2,76.60
5,Greenwich - Plumstead,Greenwich,00AL,13433,51.1,57.1,19.5,5.9,19.4,1,23.356836,8.6,55.4,41.7,14.6,16736.0,106.6,74.0,83.3,78.65
6,Hillingdon - Heathrow Villages,Hillingdon,00AS,7399,67.4,85.5,24,3.2,23.8,1.4,20.821379,9.4,50.0,43.2,17.1,12199.0,86.9,75.8,82.8,79.30
7,Lambeth - Streatham Hill,Lambeth,00AY,15107,133.6,37.9,31.8,8.5,12.2,0.5,18.320129,8.4,37.9,37.8,12.4,14263.0,57.1,76.4,79.9,78.15
8,Havering - Gooshays,Havering,00AR,12833,49.5,82,1.9,31.6,19,1.4,23.237136,12.9,12.8,10.7,2.6,14692.0,68.6,76.3,80.4,78.35
9,Croydon - Broad Green,Croydon,00AH,13213,82.2,61.3,9.3,4.2,17.2,1.3,25.761312,7.1,68.6,46.2,16.9,18652.0,83.8,75.3,83.8,79.55


## Make DF for model 

In [96]:
new_columns = ['Wardname', 'Population2011Census_x', 'Openspace', 'Carsperhousehold', 'Children', 'Greaterthan65', 'nonwhite', 'NotBorninUK', 'NotEnglishspeaking', 'GeneralFertilityRate','JobSeekers','Noqual', 'Crimerate', 'hhSocialRented',  'LifeTotal']
latest_DF = CompleteDF[new_columns]

In [97]:
latest_DF.head(1)

,Wardname,Population2011Census_x,Openspace,Carsperhousehold,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,GeneralFertilityRate,JobSeekers,Noqual,Crimerate,hhSocialRented,LifeTotal
0,Hackney - Queensbridge,11098,67.7,0.5,17.542063,8.2,44.7,38.3,11.8,69.8,13.6,24.6,61.9,57.9,79.25


## Make Model

In [62]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import label
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [98]:
# Label Encode the wards
labelEncoder = label.LabelEncoder()
latest_DF['Wardname'] = labelEncoder.fit_transform(latest_DF.Wardname)

In [99]:
latest_DF.head(1)

,Wardname,Population2011Census_x,Openspace,Carsperhousehold,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,GeneralFertilityRate,JobSeekers,Noqual,Crimerate,hhSocialRented,LifeTotal
0,190,11098,67.7,0.5,17.542063,8.2,44.7,38.3,11.8,69.8,13.6,24.6,61.9,57.9,79.25


In [100]:
# Use min max scaler
scaler = MinMaxScaler()
latest_DF_Scaled = scaler.fit_transform(latest_DF)
latest_DF_Scaled = pd.DataFrame(data=latest_DF_Scaled, columns=latest_DF.columns)

In [114]:
latest_DF_Scaled.head()

,Wardname,Population2011Census_x,Openspace,Carsperhousehold,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,GeneralFertilityRate,JobSeekers,Noqual,Crimerate,hhSocialRented,LifeTotal,IMD,_intercept
0,0.349908,0.448842,0.588761,0.000000,0.399489,0.235,0.453125,0.523659,0.114804,0.449576,0.423077,0.974227,0.167794,1.000000,0.092377,0.593028,1
1,0.867403,0.586838,0.273308,0.000000,0.438401,0.175,0.505580,0.529968,0.147029,0.305372,0.269231,0.752577,0.732070,0.662500,0.035874,0.475678,1
2,0.659300,0.253429,0.518519,0.750000,0.330011,0.250,0.251116,0.523659,0.121853,0.287465,0.016026,0.056701,0.156969,0.066071,0.114798,0.045293,1
3,0.788214,0.390301,0.770115,0.416667,0.361515,0.525,0.100446,0.440063,0.081571,0.511781,0.086538,0.432990,0.194181,0.296429,0.194619,0.204905,1
4,0.607735,0.749344,0.208174,0.000000,0.365482,0.135,0.437500,0.585174,0.126888,0.157399,0.259615,0.335052,0.652909,0.533929,0.044843,0.345235,1


### Calc IMD

In [102]:
def imd(row):
    return((row['JobSeekers']*0.55)+(row['Noqual']*0.25)+(row['Crimerate']*0.10)+(row['hhSocialRented']*0.10))

In [103]:
latest_DF_Scaled['IMD'] = latest_DF_Scaled.apply(imd, axis=1)

## Make X and Y datasets

In [118]:
y = latest_DF_Scaled['LifeTotal']
latest_DF_Scaled = latest_DF_Scaled.drop(columns=['JobSeekers', 'Noqual', 'Crimerate', 'hhSocialRented', 'LifeTotal'], axis=1)
x = latest_DF_Scaled

In [129]:
x.head()

,Wardname,Population2011Census_x,Openspace,Carsperhousehold,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,GeneralFertilityRate,IMD,_intercept
0,0.349908,0.448842,0.588761,0.000000,0.399489,0.235,0.453125,0.523659,0.114804,0.449576,0.593028,1
1,0.867403,0.586838,0.273308,0.000000,0.438401,0.175,0.505580,0.529968,0.147029,0.305372,0.475678,1
2,0.659300,0.253429,0.518519,0.750000,0.330011,0.250,0.251116,0.523659,0.121853,0.287465,0.045293,1
3,0.788214,0.390301,0.770115,0.416667,0.361515,0.525,0.100446,0.440063,0.081571,0.511781,0.204905,1
4,0.607735,0.749344,0.208174,0.000000,0.365482,0.135,0.437500,0.585174,0.126888,0.157399,0.345235,1


In [110]:
#Reference Variable
latest_DF_Scaled['_intercept'] = 1

# Train Model

In [120]:
import statsmodels.api as sm
import numpy as np
# split the dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= 0.3, random_state=0)
    
logit = sm.OLS(np.array(y_train), np.array(X_train))
    
    # Fit the model
result = logit.fit()

In [121]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     11.81
Date:                Fri, 12 Apr 2019   Prob (F-statistic):           5.48e-19
Time:                        15:21:25   Log-Likelihood:                 659.30
No. Observations:                 380   AIC:                            -1295.
Df Residuals:                     368   BIC:                            -1247.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0126      0.009      1.400      0.1